In [1]:
from IPython.display import Image, display, Math, Latex, HTML, Javascript
display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

In [2]:

# TOGGLE ON/OFF RAW CODE
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [3]:
# IMPORTS

import os
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from ipywidgets import widgets, interact, Layout, interactive, fixed
#from IPython.display import Image, display, Math, Latex, HTML, Javascript

%matplotlib inline

if os.getcwd().split('/')[-1] == 'PSpy':
    pass
else:
    paths = os.getcwd().split('/')[:-2]
    newpath = '/' + os.path.join(*paths)+ '/'
    os.chdir(newpath)


from mypysmps.io.read import read
from mypysmps.util.ps_utils import stack_ps
from mypysmps.graph.psdisplay import PSDisplay
from mypysmps.smpswidgets.widget_aux import wchooseFile, wread

concatenate = widgets.Checkbox(value=False,description='Concatenate',disabled=False,indent=False, layout = Layout(width='10%'))
filename = widgets.Dropdown(options= wchooseFile(), value = 'data/testdata/LongTerm.csv', description='Filename', layout = Layout(width='40%'))
filename2 = widgets.SelectMultiple(options= wchooseFile(), value = ['data/testdata/LongTerm.csv'], description='Filename',layout = Layout(width='50%'))

In [14]:
uif = widgets.HBox([concatenate, filename, filename2])
display(uif)

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
button = widgets.Button(description="Apply")
button.on_click(run_all)

display(button)

<IPython.core.display.Javascript object>

Button(description='Apply', style=ButtonStyle())

In [17]:
if concatenate.value:
    SMPS = read(filename.value)
    SMPS.createTimeDate()
    dunits = SMPS.datetime['units']
    smps_list = []
    date_list = []
    for afilename in filename2.value:
        asmps = read(afilename)
        asmps.createTimeDate()
        smps_list.append(asmps)
        date_list.append(asmps.datetime['data'][0])
    
    datelist = [dt.datetime.strptime(dl, dunits) for dl in date_list]
    smpslist = [x for _,x in sorted(zip(datelist,smps_list))]
    for smps in smpslist:
        SMPS = stack_ps(SMPS, smps, message = False)
else:
    SMPS = read(filename.value)
    
psd = PSDisplay(SMPS)

sample = widgets.IntSlider(min=1, max=len(SMPS.sample['data'])-1, continuous_update = False, description = 'sample #',layout = Layout(
            width = '100%'
        ))

sfields = widgets.ToggleButtons(options=SMPS.data['variables'], value = SMPS.data['variables'][0], description='Variable:', disabled=False, button_style='')
D50line = widgets.Checkbox(value=False,description='D50',disabled=False,indent=False, layout = Layout(width='20%'))
boundaryline = widgets.Checkbox(value=False,description='Boundaries',disabled=False, indent=False, layout = Layout(width='20%'))
indperiods = widgets.Checkbox(value=False,description='24 hour periods',disabled=False,indent=False, layout = Layout(width='20%'))
starttime = widgets.Text(value='00:00:00', placeholder='HH:MM:SS', description='Start time:', disabled=False)
tfields =  widgets.Dropdown(options= np.append(SMPS.data['variables'],list(SMPS.__dict__.keys())), value = 'total_concentration', description='Variable:', disabled=False, button_style='')

clim = widgets.BoundedFloatText( value=2*10**4, min=1*10**1, max=1*10**8,step=1*10**1, description='clim:',
    style = {'description_width': 'initial'}, disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))
dlim = widgets.BoundedFloatText( value=1*10**3, min=1*10**1, max=1*10**8,step=1*10**1, description='dlim:', disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))

ylim =  widgets.BoundedFloatText( value=None, min=0, max=1*10**8,step=0.001, description='ylim:', disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))

diameter = widgets.BoundedIntText( value=0, min=0, max=len(SMPS.diameter['data']),step=1, description='Diameter:', disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))

outplot =  widgets.interactive_output(psd.histogram,{'field':sfields,'sample':sample,'add_D50':D50line,'add_boundaries':boundaryline, 'xlim': dlim,'ylim':clim}) 
outplot2 = widgets.interactive_output(psd.plot, {'field':sfields,'clim':clim,'ylim':dlim, 'indicator':sample,'periods':indperiods, 'starttime':starttime})

date = widgets.interactive_output(psd.dateInfo,{'sample':sample})
#outplot3 = widgets.interactive_output(psd.timeLine, {'field':tfields,'ylim':ylim, 'indicator':sample,'periods':indperiods, 'starttime':starttime})
outplot3 = widgets.interactive_output(psd.timeLine, {'field':tfields,'ylim':ylim,'widget': fixed(True),'diameter':diameter,'indicator':sample,'periods':indperiods, 'starttime':starttime})


ui1 = widgets.HBox([sfields])
ui2 = widgets.HBox([D50line, boundaryline, clim, dlim])
uidate = widgets.HBox([date])
uiperiods = widgets.HBox([indperiods, starttime])
ui1a = widgets.HBox([tfields, ylim, diameter])
ui3 = widgets.VBox([ui1, ui2, outplot, sample, uidate, uiperiods, outplot2,ui1a, outplot3], layout = Layout(height='100%'))

                                      
display(ui3)

/home/fvanden/anaconda3/envs/aerosol/lib/python3.7/site-packages/traitlets/traitlets.py:567: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)
